# Scraping Reddit for Baltimore Stories

This script scrapes the first 100 posts in the r/baltimore subreddit and creates a dataframe

Documentation:

https://www.reddit.com/r/reddit.com/wiki/api/#wiki_read_the_full_api_terms_and_sign_up_for_usage

https://medium.com/@archanakkokate/scraping-reddit-data-using-python-and-praw-a-beginners-guide-7047962f5d29

Developer panel: https://www.reddit.com/prefs/apps

Need to revise code to address this: https://asyncpraw.readthedocs.io/en/stable/

How Reddit works: https://support.reddithelp.com/hc/en-us



In [8]:
!pip install praw
!pip install ftfy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.9 MB/s eta 0:00:00


In [3]:
import praw
import pandas as pd
import pprint
import numpy

CLIENT_ID = "XXXX"
CLIENT_SECRET = "XXXX"
USER_AGENT = "XXXXX"

reddit = praw.Reddit(
    client_id=CLIENT_ID,
    client_secret = CLIENT_SECRET,
    user_agent=USER_AGENT
)

subreddit = reddit.subreddit('baltimore')


In [9]:

# Fetch posts
posts = []
for submission in subreddit.new(limit = 100):  # Adjust the limit as needed
    posts.append([submission.id, submission.title, submission.selftext, submission.score, submission.num_comments, submission.created_utc, submission.author, submission.url])

# Convert to DataFrame
posts_df = pd.DataFrame(posts, columns=['post_id', 'title', 'body', 'score', 'num_comments', 'created_utc', 'author', 'url'])

# Fetch comments
comments = []
for submission in subreddit.new(limit=100):  # Adjust the limit as needed
    submission.comments.replace_more(limit = 0) #this limit is how many times the API will click "more comments" -- sadly, we can't do all, as we get a "too many requests" error
    for comment in submission.comments.list():
        comments.append([comment.id, comment.body, comment.score, comment.created_utc, comment.author, submission.id])

# Convert to DataFrame
comments_df = pd.DataFrame(comments, columns=['id', 'body', 'score', 'created_utc', 'author', 'post_id'])

import ftfy

# this will fix your encoding problem
posts_df['body'] = posts_df['body'].map(ftfy.fix_encoding)
comments_df['body'] = comments_df['body'].map(ftfy.fix_encoding)

merged_df = comments_df.merge(posts_df, left_on='post_id', right_on='post_id', suffixes=('_comment', '_post'))


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/l

In [13]:
merged_df.to_csv('/Users/gizmofo/Code/CompText_Jour/data/Baltimore_reddit_data.csv', index=False)

OSError: Cannot save file into a non-existent directory: 'Users/gizmofo/Code/CompText_Jour/data'

In [14]:
from google.colab import files

# First save the file
merged_df.to_csv('Baltimore_reddit_data.csv', index=False)

# Then download it
files.download('Baltimore_reddit_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
#add to file
#
import os
# add to existing CSV file
csv_file = 'Baltimore_reddit.csv'

if os.path.exists(csv_file):
    existing_posts_df = pd.read_csv(csv_file)
    combined_df = pd.concat([existing_posts_df, merged_df]).drop_duplicates(subset='id', keep='last')
    combined_df.to_csv(csv_file, index=False)
else:
    merged_df.to_csv(csv_file, index=False, encoding='utf-8')